Я помогу вам разобраться с репозиторием для обработки данных Sentinel-1. Сначала изучу все файлы, чтобы понять структуру и назначение каждого компонента.



<think duration="341" >
Теперь прочитаю остальные файлы, чтобы понять полную структуру проекта.

</think>



## Анализ структуры проекта Sentinel-1

Этот проект представляет собой комплексную систему обработки сырых радарных данных с космического аппарата Sentinel-1. Основная цель - преобразование сырых бинарных данных в радиолокационные изображения (SLC - Single Look Complex).

### 🏗️ **Общая архитектура проекта**

```
Сырые данные (.dat) → Space Packet Decoder → I/Q Corrector → SLC Processor → Радиолокационные изображения
```

### 📁 **Анализ файлов и их назначение**

#### 1. **log.py** - Система логирования
**Назначение**: Управление системой логов с ротацией файлов
- Создает лог-файлы с временными метками
- Поддерживает разные уровни логирования (INFO, WARN)
- Автоматическая ротация логов по часам

#### 2. **radarConfigurationSupportService.py** - Конфигурация радара
**Назначение**: Парсинг и обработка конфигурационных данных радара
- Извлекает параметры радара из космических пакетов
- Рассчитывает частоту дискретизации, TX Pulse Ramp Rate
- Извлекает параметры окон выборки (SWST, SWL)
- Обрабатывает BAQ (Block Adaptive Quantization) режимы

#### 3. **spDecoder.py** - Декодировщик космических пакетов
**Назначение**: Декодирование данных Space Packet протокола
- Парсит Primary Header (версия, идентификация, счетчик последовательности)
- Обрабатывает Secondary Header (службы времени, конфигурации)
- **Декодирует сжатые данные по алгоритму Хаффмана**:
  - `huffmanDecodeCore4BRC0-4` - различные схемы сжатия
  - `getIEMCode`, `getIOorQOMCode`, `getQEMCode` - декодирование I/Q компонентов
- Восстанавливает I/Q выборки из сжатых данных

#### 4. **reconstruction.py** - Восстановление выборок
**Назначение**: Восстановление значений амплитуды из декодированных кодов
- Использует таблицы нормализованных уровней восстановления
- Применяет пороговые индексы и коэффициенты сигма
- Восстанавливает аналоговые значения из цифровых кодов

#### 5. **rawDataAnalysis.py** - Анализ сырых данных
**Назначение**: Статистический анализ I/Q компонентов для калибровки
- Вычисляет средние значения (iMean, qMean)
- Рассчитывает стандартные отклонения (iSTD, qSTD)
- Определяет дисбаланс усиления I/Q (iQGain)
- Вычисляет квадратурное отклонение каналов

#### 6. **corrector.py** - Корректор данных
**Назначение**: Калибровка и исправление I/Q данных
- Загружает декодированные пакеты
- Применяет исправления на основе анализа:
  - Коррекция смещения (bias correction)
  - Коррекция дисбаланса усиления
  - Коррекция неортогональности квадратур
- Сохраняет исправленные данные

#### 7. **spCreator.py** - Создатель космических пакетов
**Назначение**: Основной конвейер обработки данных
- Считывает бинарные файлы Sentinel-1
- Создает и обрабатывает космические пакеты
- Фильтрует эхо-сигналы и режимы измерений
- Сохраняет данные по дальностным линиям (range lines)

#### 8. **slcProcessor.py** - Процессор SLC изображений
**Назначение**: Сжатие по дальности и азимуту для получения изображений
- **Range Compression**: Сжатие по дальности с использованием chirp-реплики
- **Azimuth Compression**: Сжатие по азимуту
- Генерация PNG изображений и MAT файлов
- Поддержка выделения целевых областей
- Экспорт в различные форматы

#### 9. **test.py** - Главный исполняемый файл
**Назначение**: Демонстрация полного конвейера обработки
- Загружает бинарные данные Sentinel-1
- Создает космические пакеты (`SPCreator`)
- Применяет коррекцию (`Corrector`) 
- Генерирует изображения (`SLCProcessor`)

### 🔄 **Поток обработки данных**

1. **Загрузка**: `BinFile` → `SPCreator`
2. **Декодирование**: `SPDecoder` → извлечение I/Q компонентов
3. **Восстановление**: `reconstruction` → аналоговые значения
4. **Анализ**: `RawDataAnalysis` → статистические параметры
5. **Коррекция**: `Corrector` → калибровка данных
6. **Сжатие**: `SLCProcessor` → радиолокационные изображения

### 🎯 **Ключевые особенности для изучения**

- **Space Packet Protocol**: Стандарт космической связи CCSDS
- **Block Adaptive Quantization (BAQ)**: Сжатие данных на борту спутника
- **Huffman Coding**: Энтропийное сжатие для I/Q компонентов
- **Chirp Processing**: Радиолокационная обработка сигналов
- **Range/Azimuth Compression**: Формирование радиолокационных изображений

Этот проект предоставляет полный цикл обработки от сырых бинарных данных до готовых радиолокационных изображений, что делает его отличным ресурсом для изучения обработки данных дистанционного зондирования.

# Полный анализ проекта обработки данных Sentinel-1

## 📋 Содержание
1. [Обзор проекта](#обзор-проекта)
2. [Архитектура системы](#архитектура-системы)
3. [Подробный анализ компонентов](#подробный-анализ-компонентов)
4. [Поток обработки данных](#поток-обработки-данных)
5. [Ключевые алгоритмы](#ключевые-алгоритмы)
6. [Использование и примеры](#использование-и-примеры)

---

## 🔍 Обзор проекта

Этот проект представляет собой комплексную систему обработки сырых радарных данных с космического аппарата Sentinel-1. Основная цель - преобразование бинарных данных в радиолокационные изображения (SLC - Single Look Complex).

**Технологический стек:**
- Python 3.x
- NumPy (математические вычисления)
- Matplotlib (визуализация)
- SciPy (цифровая обработка сигналов)
- Pickle (сериализация данных)
- Multiprocessing (параллельная обработка)

**Входные данные:** 
- Бинарные файлы формата .dat, полученные с Copernicus Data Space
- Сырые Space Packet данные с компрессией BAQ

**Выходные данные:**
- Радиолокационные изображения в формате PNG
- MAT файлы для дальнейшего анализа
- Лог-файлы процесса обработки

---

## 🏗️ Архитектура системы

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   Бинарные      │    │  Space Packet   │    │  Восстановление │
│   данные        │───▶│   Декодер       │───▶│  выборок        │
│   (.dat)        │    │                 │    │                 │
└─────────────────┘    └─────────────────┘    └─────────────────┘
                                │                       │
                                ▼                       ▼
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│  Анализ и       │    │  I/Q Корректор  │    │  SLC Процессор  │
│  Калибровка     │◀───│                 │───▶│                 │
│                 │    │                 │    │                 │
└─────────────────┘    └─────────────────┘    └─────────────────┘
         ▲                                        │
         └─────────────────────────────────────────┘
                            │
                            ▼
                   ┌─────────────────┐
                   │  Радиолокационные│
                   │  изображения     │
                   │  (PNG, MAT)      │
                   └─────────────────┘
```

---

## 📁 Подробный анализ компонентов

### 1. `__init__.py`
**Назначение:** Инициализация пакета `spacePacket`

**Содержимое:** Минимальный файл для Python-пакета

---

### 2. `log.py` - Система логирования

#### Класс `Logger`
**Глобальные переменные:**
- `_logDict` (dict): Словарь логгеров
- `logDir` (str): Путь к директории логов  
- `log0` (Path): Объект Path для создания директории

#### Функции:

**`makeLogger(fileName)`**
- **Назначение:** Создание логгера с ротацией
- **Параметры:** 
  - `fileName` (str): Имя файла лога
- **Функциональность:**
  - Устанавливает уровень логирования INFO
  - Создает ротационный файл-хендлер с ежечасной ротацией
  - Определяет формат: `время|уровень|файл|строка|сообщение`
  - Сохраняет логгер в словаре

**`getLogger(moduleName)`**
- **Назначение:** Получение или создание логгера
- **Параметры:**
  - `moduleName` (str): Имя модуля
- **Возвращает:** Объект логгера
- **Логика:** Создает уникальное имя файла с временной меткой

---

### 3. `radarConfigurationSupportService.py` - Служба поддержки конфигурации радара

#### Класс `RadarConfigurationSupportService`
**Переменные экземпляра:**
- `fref` (float): Базовая частота 37.53472224 МГц

#### Методы:

**`getSamplingFrequency(data)`**
- **Назначение:** Расчет частоты дискретизации после децимирования
- **Параметры:** `data` (int): Индекс для коэффициентов децимирования
- **Возвращает:** Частоту дискретизации в МГц
- **Коэффициенты:** `[3, 8/3, NaN, 20/9, 16/9, 3/2, 4/3, 2/3, 12/7, 5/4, 6/13, 16/11]`

**`getTXPRR(data)`**
- **Назначение:** Расчет скорости нарастания TX импульса
- **Параметры:** `data` (bytes): 2 байта данных
- **Возвращает:** TXPRR в МГц/мкс
- **Формула:** `(-1)^sign * value * (fref^2 / 2^21)`

**`getTXPSF(data, txprr)`**
- **Назначение:** Расчет начальной частоты TX импульса
- **Параметры:** 
  - `data` (bytes): 2 байта
  - `txprr` (float): Скорость нарастания
- **Формула:** `txprr / (4 * fref) + (-1)^sign * value * fref / 2^14`

**`getTXPL(data)`**
- **Назначение:** Расчет длительности TX импульса
- **Параметры:** `data` (bytes): 3 байта
- **Возвращает:** TXPL в микросекундах

**`parseData(data, packet)`**
- **Назначение:** Парсинг всех параметров конфигурации
- **Параметры:**
  - `data` (bytes): 28 байт данных RCSS
  - `packet` (SpacePacket): Объект пакета для сохранения
- **Извлекаемые параметры:**
  - `errorFlag`: Флаг ошибки
  - `BAQMode`: Режим BAQ (5 бит)
  - `baqBlockLength`: Длина блока BAQ
  - `SamplingFrequencyAfterDecimation`: Частота после децимирования
  - `TXPRR`: Скорость нарастания импульса
  - `TXPSF`: Начальная частота импульса
  - `TXPL`: Длительность импульса
  - `SWST`: Время начала окна выборки
  - `SWL`: Длина окна выборки
  - `signalType`: Тип сигнала (4 бит)
  - `swathNumber`: Номер свипа

---

### 4. `spDecoder.py` - Декодировщик космических пакетов

#### Класс `SPDecoder`
**Переменные экземпляра:**
- `binFile` (BinFile): Объект бинарного файла
- `hCode` (int): Буфер для обработки битов
- `hCodeBitsSize` (int): Размер буфера в битах
- `userDataCnt` (int): Счетчик пользовательских данных
- `rcss` (RadarConfigurationSupportService): Сервис конфигурации
- `huffmanDecodeFuncDict` (dict): Словарь функций декодирования Хаффмана

#### Методы:

**`preparePacketPrimaryHeader(packet)`**
- **Назначение:** Парсинг основного заголовка пакета
- **Возвращает:** True/False (успех операции)
- **Извлекает:**
  - `packetVersionNumber`: Версия пакета (3 бита)
  - `packetIdentification`: Идентификация (16 бит)
  - `sequenceFlags`: Флаги последовательности (2 бита)
  - `packetSequenceCount`: Счетчик последовательности (14 бит)
  - `packetDataLength`: Длина данных пакета

**`preparePacketSecondaryHeader(packet)`**
- **Назначение:** Парсинг вторичного заголовка
- **Компоненты:**
  - `datationService`: Служба времени (6 байт)
  - `parseFixedAncillaryDataField`: Фиксированные вспомогательные данные
  - `subCommutationAncillaryDataServiceField`: Вспомогательные данные (3 байт)
  - `countersService`: Счетчики (8 байт)
  - Конфигурация радара (28 байт)
  - `RadarSampleCountService`: Количество выборок (2 байт)

**`parseFixedAncillaryDataField(packet)`**
- **Извлекает:**
  - `syncMarker`: Маркер синхронизации (4 байт)
  - `dataTakeID`: ID съемки (4 байт)
  - `eccNumber`: Номер ECC (1 байт)
  - `testMode`: Тестовый режим (3 бита из 1 байт)
  - `instrumentConfigurationID`: ID конфигурации (4 байт)

**`prepareUserDataFiled(packet)`**
- **Назначение:** Декодирование пользовательских данных
- **Алгоритм:**
  1. Декодирование IE кодов Хаффмана
  2. Декодирование IO кодов Хаффмана  
  3. Декодирование QE кодов Хаффмана
  4. Декодирование QO кодов Хаффмана
  5. Восстановление значений выборок
  6. Формирование I/Q массивов

**`getIEMCode(packet)`**
- **Назначение:** Декодирование I-E компонентов
- **Логика:** Обрабатывает данные блоками по 128 выборок

**`getIOorQOMCode(packet)`**
- **Назначение:** Декодирование I-O или Q-O компонентов

**`getQEMCode(packet)`**
- **Назначение:** Декодирование Q-E компонентов
- **Особенность:** Включает пороговые индексы

**`huffmanDecode(mCodeQuantity, bitRateCode)`**
- **Назначение:** Основная функция декодирования Хаффмана
- **Выбирает:** Соответствующую функцию декодирования по BRC

#### Функции декодирования Хаффмана:

**`huffmanDecodeCore4BRC0()` - BRC = 0**
- **Коды:**
  - `0` → код 0
  - `10` → код 1
  - `110` → код 2
  - `111` → код 3

**`huffmanDecodeCore4BRC1()` - BRC = 1**
- **Коды:**
  - `0` → код 0
  - `10` → код 1
  - `110` → код 2
  - `1110` → код 3
  - `1111` → код 4

**`huffmanDecodeCore4BRC2()` - BRC = 2**
- **Дополнительно:**
  - `11110` → код 4
  - `111110` → код 5
  - `111111` → код 6

**`huffmanDecodeCore4BRC3()` - BRC = 3**
- **Двухбитный анализ** для оптимизации
- Коды 0-9

**`huffmanDecodeCore4BRC4()` - BRC = 4**
- **Максимальное сжатие** (15 кодов)
- Использует многоуровневый анализ

**Вспомогательные методы:**
- `interceptUserDataBits(num)`: Извлечение бит из потока
- `fillHCodeByBinFile()`: Заполнение буфера из файла
- `remainHCode(num)`: Обновление остатка буфера
- `alignData(bitsLength)`: Выравнивание данных

---

### 5. `reconstruction.py` - Восстановление выборок

#### Функция `sampleReconstruction(brc, thidx, sign, mCode)`
**Назначение:** Восстановление аналогового значения из цифрового кода
**Параметры:**
- `brc` (int): Bit Rate Code (0-4)
- `thidx` (int): Индекс порога (0-8)
- `sign` (int): Знак (+1 или -1)
- `mCode` (int): M-код (значение от 0 до максимума для BRC)

**Логика восстановления:**
1. **Простое восстановление** (thidx ≤ thidxSimpleFlag):
   - Если mCode < mCodeFlag → `res = sign * mCode`
   - Иначе → использует `fDBQASimpleReconstructionParam`

2. **Нормализованное восстановление** (thidx > thidxSimpleFlag):
   - `res = sign * normalisedReconstructionLevels[mCode][brc] * sigmaFactors[thidx]`

#### Таблицы параметров:

**`fDBQASimpleReconstructionParam`**
- Матрица 9×5 для простого восстановления
- Значения зависят от brc и thidx

**`normalisedReconstructionLevels`**
- Матрица 16×5 нормализованных уровней
- Используется для сложного восстановления

**`sigmaFactors`**
- Массив 128 коэффициентов сигма
- Зависит от thidx (0-127)

---

### 6. `rawDataAnalysis.py` - Анализ сырых данных

#### Класс `RawDataAnalysis`
**Переменные экземпляра:**
- `spacePackets` (list): Список космических пакетов
- `allDataLength` (int): Общая длина данных
- `iMean` (float): Среднее значение I-компонента
- `qMean` (float): Среднее значение Q-компонента
- `iSTD` (float): Стандартное отклонение I
- `qSTD` (float): Стандартное отклонение Q
- `iQGain` (float): Коэффициент усиления I/Q
- `lowerBounds`, `upperBounds` (float): Границы для iQGain
- `quadratureDeparture` (float): Квадратурное отклонение

#### Методы:

**`process()`**
- Запускает полный анализ в последовательности:
  1. `calcMean()` - расчет средних
  2. `calcSTD()` - расчет стандартных отклонений
  3. `calcIQGain()` - расчет дисбаланса усиления
  4. `calcIQquadraturedeparture()` - расчет неортогональности

**`calcMean()`**
- **Алгоритм:** Суммирование всех значений / количество
- **Обновляет:** `iMean`, `qMean`
- **Сложность:** O(n), где n - общее количество выборок

**`calcSTD()`**
- **Формула:** `sqrt(sum((x - mean)^2) / n)`
- **Обновляет:** `iSTD`, `qSTD`
- **Использует:** Предварительно вычисленные средние

**`calcIQGain()`**
- **Формула:** `iQGain = iSTD / qSTD`
- **Границы:** `1 ± 3/sqrt(allDataLength)`
- **Проверка:** Контроль разумности коэффициента

**`calcIQquadraturedeparture()`**
- **Алгоритм:** Вычисление корреляции I/Q каналов
- **Формула ck:** `siq / sqrt(sii * sqq)`
- **Преобразование Fisher:** `zk = 0.5 * log((1+ck)/(1-ck))`
- **Квадратурное отклонение:** `arcsin(c)`, где `c = tanh(zMean)`
- **Валидация:** Ограничение границами неопределенности

---

### 7. `corrector.py` - Корректор данных

#### Класс `Corrector`
**Переменные экземпляра:**
- `polar` (str): Поляризация ('VV', 'VH')
- `name` (str): Имя файла
- `packets` (list): Список исправленных пакетов
- `analysis` (RawDataAnalysis): Объект анализа

#### Методы:

**`correct()`**
- **Основной метод коррекции:**
  1. `preparePackets()` - загрузка декодированных данных
  2. `IQBias()` - применение I/Q коррекций
  3. `saveCorrectedData()` - сохранение результатов

**`preparePackets()`**
- **Загружает:** Декодированные пакеты из pickle файла
- **Инициализирует:** `RawDataAnalysis` и выполняет анализ
- **Создает:** Анализ статистических параметров

**`IQBias()`**
- **1. Коррекция смещения:**
  ```python
  packet.ISampleValue += iMean
  packet.QSampleValue += qMean
  ```
- **2. Коррекция усиления:**
  ```python
  packet.QSampleValue = iQGain * packet.QSampleValue
  ```
- **3. Коррекция неортогональности:**
  ```python
  packet.QSampleValue = packet.QSampleValue / cos(quadratureDeparture) - 
                       packet.ISampleValue * tan(quadratureDeparture)
  ```

**`saveCorrectedData()`**
- **Формат:** Сохранение в pickle формат
- **Путь:** `./data/{polar}/correct/{name}`

---

### 8. `spCreator.py` - Создатель космических пакетов

#### Класс `SPCreator`
**Переменные экземпляра:**
- `binFile` (BinFile): Объект бинарного файла
- `spacePacketsLengthMAX` (int): Максимальное количество пакетов
- `startIndex` (int): Начальный индекс (не используется)

#### Методы:

**`createSpacekets()`**
- **Основной цикл обработки:**
  - Создает `SPDecoder` для каждого пакета
  - Парсит Primary Header
  - Парсит Secondary Header  
  - Фильтрует эхо-сигналы и режимы измерений
  - Декодирует пользовательские данные
  - Группирует пакеты по свипам
  - Сохраняет данные по дальностным линиям

**`saveOneRangeData(packetIndex, packets)`**
- **Назначение:** Сохранение данных одной дальности
- **Формат:** Pickle файл `rangeLine_{packetIndex}.pkl`
- **Критерий сохранения:** Смена номера свипа

**Условия фильтрации:**
```python
if (not packet0.isEcho()) or (not packet0.isMeasurementMode()):
    # Пропускаем не-эхо или не-измерительные данные
```

---

### 9. `slcProcessor.py` - Процессор SLC изображений

#### Класс `SLCProcessor`
**Переменные экземпляра:**
- `polar` (str): Поляризация
- `name` (str): Имя файла

#### Основные методы:

**`compress()`**
- **Главный метод обработки:**
  1. Загружает скорректированные пакеты
  2. Выполняет `rangeCompress()` - сжатие по дальности
  3. Выполняет `azimuthCompressTarget()` - сжатие по азимуту

**`rangeCompress(packets)`**
- **Алгоритм:** Корреляция эхо-сигнала с chirp-репликой
- **Шаги:**
  1. Создание chirp-реплики:
     ```python
     phi1 = TXPSF + TXPRR * TXPL / 2
     phi2 = TXPRR / 2
     chirp = exp(-1j*2*pi*(phi1*t + phi2*t^2))
     ```
  2. Формирование комплексных данных:
     ```python
     echoData = ISampleValue + 1j*QSampleValue
     ```
  3. Корреляция:
     ```python
     result = correlate(chirp, echoData, mode='full')
     ```

**`azimuthCompressTarget(rangeCompressMat, targetAreas)`**
- **Назначение:** Сжатие по азимуту для целевых областей
- **Алгоритм:**
  1. Извлечение azimutal chirp из данных
  2. Применение корреляции по азимуту
  3. Фокусировка на заданных областях

**Методы визуализации:**

**`showALL(tempMatLen, bias, tempMat)`**
- Сохранение полного изображения
- Путь: `./pic/{polar}/all/{name}_all_{tempMatLen}_{bias}_target.png`
- Сохранение в MAT формате

**`showPart(tempMat)`**
- Разделение на части для детального анализа
- Разбиение по строкам (60 частей)

**`saveFig(data, figName, vmax0)`**
- **Сохранение изображений:**
  - MAT файл (scipy.io.savemat)
  - PNG изображение с масштабированием
  - Настройка размера и DPI

**Методы анализа сырых данных:**
- `saveOrigin()`: Сохранение исходных I/Q данных
- `saveOriginSingle()`: Сохранение отдельных I и Q компонентов

---

### 10. `spacePacket.py` - Класс космического пакета

#### Класс `SpacePacket`
**Переменные для декодирования:**
- `hCode`, `hCodeBitsSize`, `userDataCnt`: Состояние битового потока
- `bitRateCodeList`, `thresholdIndexList`: Списки кодов

**Переменные заголовков:**
- `primaryHeader` (bytes): Основной заголовок (4 байт)
- `packetDataLength` (int): Длина данных пакета
- `datationService` (bytes): Служба времени (6 байт)
- `fixedAncillaryDataField` (bytes): Фиксированные данные
- `subCommutationAncillaryDataServiceField` (bytes): Вспомогательные данные
- `countersService` (bytes): Счетчики (8 байт)
- `radarSampleCountService` (bytes): Количество выборок

**Переменные данных:**
- `ISampleValue`, `QSampleValue` (list/array): I/Q выборки

**Методы проверки:**
- `isEcho()`: Проверка эхо-сигнала (signalType == 0)
- `isMeasurementMode()`: Проверка измерительного режима (testMode == 0)

**Методы сохранения:**
- `saveTHIDX(thidx)`: Сохранение индекса порога
- `saveBRC(brc)`: Сохранение Bit Rate Code

---

### 11. `binFile.py` - Класс бинарного файла

#### Класс `BinFile`
**Переменные экземпляра:**
- `file` (file): Объект файла
- `polar` (str): Поляризация (извлекается из имени файла)
- `size` (int): Размер файла в байтах

**Конструктор `__init__(filePath)`**
- **Открывает:** Бинарный файл в режиме чтения
- **Извлекает:** Поляризацию из имени файла (позиции 13-15)
- **Получает:** Размер файла

**Метод `read(n)`**
- **Назначение:** Чтение n байт из файла
- **Параметры:** `n` (int): Количество байт
- **Возвращает:** Байтовую строку

---

### 12. `test.py` - Главный исполняемый файл

#### Основные функции:

**`ReadFile()`**
- **Назначение:** Тестовое чтение бинарного файла
- **Показывает:** Первые 10 байт данных
- **Отладочная информация:** Анализ структуры данных

**`azimuthCompress(rangeCompressMat)`**
- **Демонстрация:** Алгоритма сжатия по азимуту
- **Визуализация:** С помощью matplotlib
- **Параметры:** Размер окна 300 отсчетов

**`showFreqImg()`**
- **Назначение:** Частотный анализ данных
- **Обработка:** FFT преобразование с маскированием
- **Выход:** Частотно-временное изображение

#### Основной блок `if __name__ == "__main__"`
**Полный конвейер обработки:**
1. **Инициализация:**
   ```python
   filePath = './data/archives/S1A_IW_RAW__...dat'
   file0 = BinFile(filePath)
   ```

2. **Создание космических пакетов:**
   ```python
   creator = SPCreator(file0)
   creator.createSpacekets()
   ```

3. **Обработка файлов:**
   ```python
   for name in fileList0:
       corrector = Corrector(polar, name)
       corrector.correct()
       
       slcProcess = SLCProcessor(polar, name)
       slcProcess.compress()
   ```

---

## 🔄 Поток обработки данных

### Этап 1: Инициализация и загрузка
```
BinFile → SPCreator → SPDecoder → SpacePacket
```

### Этап 2: Декодирование Space Packet
```
Primary Header → Secondary Header → User Data Field
    ↓                ↓                     ↓
Версия/ID        RCSS/Анcil      Huffman → Reconstruction
```

### Этап 3: Анализ и коррекция
```
RawDataAnalysis → I/Q Bias → I/Q Gain → I/Q Quadrature
```

### Этап 4: Формирование изображений
```
Range Compress → Azimuth Compress → Visualization
```

---

## 🧮 Ключевые алгоритмы

### 1. Алгоритм Хаффмана для BAQ
**Цель:** Эффективное сжатие I/Q данных
**Режимы:** 5 различных BRC (Bit Rate Code)
**Таблицы кодов:**
- BRC=0: 4 кода (0→0, 10→1, 110→2, 111→3)
- BRC=4: 15 кодов (максимальное сжатие)

### 2. Восстановление аналоговых значений
```python
if thidx ≤ thidxSimpleFlag:
    res = sign * mCode  # Простое
else:
    res = sign * normalisedReconstructionLevels[mCode][brc] * sigmaFactors[thidx]  # Сложное
```

### 3. I/Q калибровка
```python
# 1. Коррекция смещения
I += iMean
Q += qMean

# 2. Коррекция усиления  
Q *= iQGain

# 3. Коррекция неортогональности
Q = Q / cos(quad) - I * tan(quad)
```

### 4. Range Compression
```python
# Chirp генерация
phi1 = TXPSF + TXPRR * TXPL / 2
phi2 = TXPRR / 2
chirp = exp(-1j*2*pi*(phi1*t + phi2*t**2))

# Корреляция
result = correlate(chirp, echo, mode='full')
```

### 5. Azimuth Compression
```python
# Извлечение chirp из данных
maxPos = find_maximum(rangeCompressed)
azimuth_chirp = extract_chirp(rangeCompressed, maxPos)

# Применение по всем дальностям
for range_bin in all_ranges:
    azimuth_filtered[range_bin] = correlate(azimuth_chirp, rangeCompressed[range_bin])
```

---

## 📊 Форматы данных

### Входные файлы
- **.dat файлы:** Сырые Space Packet данные
- **Формат:** Бинарные данные с BAQ сжатием
- **Источник:** Copernicus Data Space

### Промежуточные файлы
- **rangeLine_*.pkl:** Декодированные пакеты по дальностям
- **pickle формат:** Python сериализация

### Выходные файлы
- **PNG изображения:** Радиолокационные изображения
- **MAT файлы:** Данные для MATLAB/Octave анализа
- **Лог файлы:** Детальная информация о процессе

---

## 🎯 Использование и примеры

### Базовое использование:
```python
# 1. Открытие файла
bin_file = BinFile('path/to/data.dat')

# 2. Создание пакетов  
creator = SPCreator(bin_file)
creator.createSpacekets()

# 3. Коррекция
corrector = Corrector(polar, 'rangeLine_123.pkl')
corrector.correct()

# 4. Формирование изображения
processor = SLCProcessor(polar, 'rangeLine_123.pkl')
processor.compress()
```

### Анализ конкретной области:
```python
# Выделение целевой области
target_areas = np.array([[10600, 10690, 650, 1050]])
processor.azimuthCompressTarget(range_data, target_areas)
```

### Визуализация результатов:
```python
processor.saveFig(data, './output/image', vmax=7e6)
processor.showPart(data)
```

---

## 🔧 Настройка и конфигурация

### Параметры обработки:
- **Максимум пакетов:** `spacePacketsLengthMAX = 2000000`
- **Размер окна chirp:** `tempMatLen = 620`
- **Масштабирование изображений:** `scale = 100`
- **DPI:** `dpi = 128`

### Пути данных:
- **Входные:** `./data/archives/`
- **Декодированные:** `./data/{polar}/decode/`
- **Скорректированные:** `./data/{polar}/correct/`
- **Изображения:** `./pic/{polar}/all/` и `./pic/{polar}/part/`
- **Логи:** `./log/`

---

## ⚠️ Важные замечания

### Требования к системе:
- Python 3.x с NumPy, Matplotlib, SciPy
- Достаточно оперативной памяти для обработки больших массивов
- Дисковое пространство для промежуточных файлов

### Ограничения:
- Обработка по одному файлу за раз
- Необходимость значительного дискового пространства
- Длительное время обработки для больших изображений

### Рекомендации:
- Использование SSD для ускорения работы
- Мониторинг использования памяти
- Регулярная очистка промежуточных файлов

---

## 📚 Технические детали

### Space Packet Protocol (CCSDS):
- **Primary Header:** 6 байт (версия, идентификация, последовательность)
- **Secondary Header:** Переменная длина (службы, конфигурация)
- **User Data:** Сжатые I/Q компоненты с BAQ

### BAQ (Block Adaptive Quantization):
- **Размер блока:** 128 выборок
- **BRC режимы:** 5 уровней сжатия
- **Пороговые индексы:** 0-8 уровней квантования

### Обработка сигналов:
- **Частоты:** L-Band (1.2 ГГц)
- **Поляризации:** VV, VH
- **Разрешение:** По дальности и азимуту
- **Динамический диапазон:** Адаптивное квантование

---

**Дата создания:** 2025-11-12  
**Версия анализа:** 1.0  
**Автор:** MiniMax Agent

## RawDataAnalysis  
 Этот файл — не просто набор статистических функций, а фундаментальный инструмент для **калибровки и оценки качества радарного приёмника**.

### Физический контекст: Что такое I/Q-сигнал?

Сигнал, который принимает радар, является аналоговым и имеет высокую частоту. Чтобы его оцифровать, его не просто записывают, а сначала **понижают частоту** (down-convert). В результате этой операции высокочастотный сигнал превращается в **комплексный сигнал**, состоящий из двух компонент:

*   **I (In-phase):** Синфазная компонента. Это, по сути, амплитуда сигнала, умноженная на `cos(ωt)`.
*   **Q (Quadrature):** Квадратурная компонента. Это амплитуда сигнала, умноженная на `sin(ωt)`.

В идеальном мире I и Q — это две проекции одного и того же вектора сигнала, сдвинутые по фазе ровно на 90°. Любые отклонения от этого идеала — это **аппаратные ошибки** (усилителей, смесителей, АЦП), которые искажают сигнал. Код в `rawDataAnalysis.py` как раз и измеряет эти ошибки.

---

### Математический и физический анализ функций

#### 1. `calcMean()`: Вычисление DC Offset (постоянной составляющей)

*   **Математика:** Вычисляется среднее арифметическое: `μ = (1/N) * Σ(x_i)`.
*   **Физический смысл:** Для идеального радиолокационного сигнала, который центрирован вокруг нуля, среднее значение должно быть равно нулю. Если оно не равно нулю, это означает **DC offset** (постоянное смещение).
*   **Причина возникновения:** Смещение может появиться из-за несовершенства аналоговых компонентов или смещения в АЦП (аналого-цифровом преобразователе).
*   **Зачем это нужно:** Знание DC offset необходимо для его последующей компенсации. Если его не убрать, он будет проявляться на изображении как постоянный фон или артефакты.

#### 2. `calcSTD()`: Вычисление Стандартного Отклонения (Уровень Мощности Сигнала)

*   **Математика:** Вычисляется стандартное отклонение: `σ = sqrt( (1/N) * Σ(x_i - μ)² )`.
*   **Физический смысл:** Стандартное отклонение для случайного сигнала (а шумовая часть SAR-сигнала очень похожа на случайный) является мерой его мощности. `std(I)` и `std(Q)` показывают среднюю мощность сигнала в I и Q каналах.
*   **Зачем это нужно:** Это базовая характеристика сигнала. Она используется для нормализации и для сравнения мощностей в двух каналах, что напрямую ведет к следующей функции.

#### 3. `calcIQGain()`: Измерение Дисбаланса Усиления (Gain Imbalance)

*   **Математика:** Вычисляется отношение стандартных отклонений: `Gain = std(I) / std(Q)`.
*   **Физический смысл:** В идеале усиление в I и Q каналах должно быть одинаковым, поэтому `std(I) ≈ std(Q)` и их отношение должно быть близко к **1**. Если отношение отличается от 1, это означает, что один канал усиливается сильнее другого. Это **дисбаланс усиления**.
*   **Причина возникновения:** Различия в параметрах усилителей для I и Q трактов, допуски на компоненты.
*   **Зачем это нужно:** Дисбаланс усиления искажает амплитуду итогового комплексного сигнала. Зная это отношение, можно ввести корректирующий множитель для одного из каналов, чтобы выровнять их усиление.

#### 4. `calcIQquadraturedeparture()`: Измерение Фазового Дисбаланса (Phase Imbalance)

Это самая сложная, но и самая важная функция.

*   **Математика:**
    1.  Вычисляется **коэффициент корреляции Пирсона** (`ck`) между I и Q компонентами.
    2.  Применяется **Z-преобразование Фишера** (`zk = 0.5 * ln((1+ck)/(1-ck))`). Это делается для того, чтобы стабилизировать дисперсию коэффициента корреляции и сделать его более удобным для статистического анализа.
    3.  Результат преобразуется обратно в меру фазовой ошибки через `asin(c)`.
*   **Физический смысл:**
    *   В идеале I и Q компоненты сигнала должны быть **некоррелированы** (статистически независимы). Их коэффициент корреляции `ck` должен быть близок к **0**.
    *   Однако, если фазовый сдвиг между I и Q **отличается от 90°**, между ними появляется корреляция.
        *   Если сдвиг **меньше 90°**, корреляция будет **положительной**.
        *   Если сдвиг **больше 90°**, корреляция будет **отрицательной**.
    *   Функция `asin(c)` как раз и переводит эту корреляцию в меру **фазового отклонения** от идеальных 90° (в радианах).
*   **Причина возникновения:** Несовершенство аналоговых смесителей, которые создают 90-градусный сдвиг.
*   **Зачем это нужно:** Фазовый дисбаланс — одна из самых серьезных ошибок. Она приводит к тому, что на итоговом изображении появляется "зеркальный" сигнал (image frequency), который маскирует реальный сигнал. Зная величину фазовой ошибки, можно применить сложную математическую коррекцию, чтобы устранить этот артефакт.

### Итог

Класс `RawDataAnalysis` — это **система диагностики радарного приёмника**. Он не обрабатывает данные для создания изображения, а измеряет, насколько "больной" сам приёмник. Результатом его работы являются **калибровочные параметры**:
1.  `iMean`, `qMean` (DC offset)
2.  `iQGain` (Gain Imbalance)
3.  `quadratureDeparture` (Phase Imbalance)

Эти параметры затем передаются в следующий блок обработки (например, `corrector`), который, используя их, вносит поправки в сырые данные, чтобы компенсировать аппаратные искажения. Без этого шага невозможно получить качественное и точное SAR-изображение.

# LEVEL 1



Это отличный и очень важный вопрос. Реализация коррекции EAP (Elevation Antenna Pattern) требует внимательного подхода, так как ошибка на этом этапе приведет к радиометрическим искажениям (неравномерной яркости изображения по дальности) и, что более важно, к фазовым ошибкам, критичным для интерферометрии.

Ниже приведен подробный разбор алгоритма на основе предоставленных документов, с пояснением каждого этапа.

### 1. Суть задачи (Что мы делаем и зачем)

Алгоритм 9.5.1 EAP Correction призван скомпенсировать влияние диаграммы направленности антенны (ДНА) на сигнал.
*   Антенна Sentinel-1 имеет разный коэффициент усиления в зависимости от угла визирования (угла места).
*   В файле `aux_cal.xml` (или `aux_cor.xml` в ваших файлах) хранится эта "двухсторонняя" диаграмма направленности (2-way EAP) как комплексная величина.
*   **Комплексная величина** — это ключевой момент. Ранее (IPF версии ниже 2.43) использовалась только амплитуда (усиление). С версии IPF 2.43 используется и фаза.
*   **Цель**: Разделить исходные данные SAR ($s$) на комплексную ДНА ($G_{eap}$), чтобы получить скорректированные данные ($s_{corr}$), где влияние антенны устранено.

Формула коррекции:
$$s_{corr}(\eta, \tau) = \frac{s(\eta, \tau)}{G_{eap}(\theta)} = \frac{s(\eta, \tau)}{|G_{eap}(\theta)| \cdot e^{j\Phi_{eap}(\theta)}}$$

Где:
*   $s$ — исходный комплексный пиксель (SLC).
*   $G_{eap}$ — комплексное значение ДНА для данного угла.
*   $\eta$ — азимутальное время.
*   $\tau$ — время по дальности (slant range time).

---

### 2. Входные данные и переменные

Для реализации вам понадобятся данные из трех источников:

**А. AUX_CAL (Вспомогательный файл калибровки)**
На основе файла `таблица.docx` и PDF с описанием фазы:
*   `elevationAntennaPattern/values`: Вектор комплексных чисел (I, Q), описывающих форму ДНА. Это сама функция $G_{eap}$.
*   `elevationAntennaPattern/elevationAngleIncrement`: Шаг угла ($\Delta\theta$), обычно 0.05 градуса.
*   `swath` и `polarisation`: Идентификаторы для выбора правильной ДНА (IW1/IW2..., HH/HV...).

**Б. Метаданные продукта (Safe-файл)**
*   `azimuthTime`: Время захвата (для расчета угла крена).
*   `slantRangeTime`: Время запаздывания сигнала для каждого пикселя по дальности.
*   `ascendingNodeTime`: Время восходящего узла орбиты (для расчета позиции).

**В. Закон управления креном (Roll Steering Law)**
Описан в документе `Sentinel-1-IPF_EAP_Phase_correction` (Раздел 7). Он нужен, чтобы понять, куда именно смотрела антенна в момент съемки.

---

### 3. Пошаговое описание алгоритма

Алгоритм можно разбить на 5 ключевых этапов.

#### Этап 1: Загрузка и подготовка EAP из AUX_CAL

В файле `aux_cal.xml` ДНА хранится как вектор значений, привязанных к углам в **системе координат антенны** ($\theta_{AM}$).

1.  Прочитайте вектор `values` (комплексные числа I, Q).
2.  Прочитайте шаг угла `elevationAngleIncrement` ($\Delta\theta$).
3.  Постройте вектор углов $\theta_{AM}$, соответствующих каждому элементу вектора значений.
    Согласно документу (стр. 9 PDF-файла, Step 5.2), вектор строится симметрично относительно нуля (или центрального элемента):
    $$\theta_{AM} = \left[ -\frac{N_{elt}-1}{2}, \dots, 0, \dots, \frac{N_{elt}-1}{2} \right] \times \Delta\theta$$
    Где $N_{elt}$ — количество элементов в векторе `values`.

    *Теперь у вас есть функция $G_{eap}(\theta_{AM})$.*

#### Этап 2: Расчет угла крена (Roll Steering)

Спутник наклоняется ("кренится"), чтобы保持 луч перпендикулярно поверхности Земли по мере изменения высоты орбиты. Этот угол меняется со временем.

1.  Определите время от восходящего узла орбиты ($\Delta t_{anx}$).
    $$\Delta t_{anx} = \text{ВремяБурста} - \text{AscendingNodeTime}$$
2.  Рассчитайте текущую высоту спутника ($H$) как функцию времени от ANX (используя формулы из Раздела 7.2 PDF: синусоидальная модель).
3.  Рассчитайте номинальный угол крена ($\theta_{roll}$) на основе высоты (формула из Раздела 7.1 PDF).
    $$\theta_{roll} \approx \theta_{ref} - \alpha_{roll} \cdot (H - H_{ref})$$
    *Эти функции объединены в `anx2roll`.*

#### Этап 3: Определение угла визирования для каждого пикселя

Для каждого пикселя по дальности нужно узнать его угол места в **геометрии визирования** ($\theta_{GV}$), то есть угол между спутником и целью относительно земной поверхности.

*   Если вы реализуете пост-процессинг (используя готовый продукт): В файле `Sentinel-1-IPF_EAP_Phase_correction` (Раздел 3.2 и 5.5) сказано, что в аннотациях продукта часто уже есть вектор `elevationAngle` ($\theta_{eap}$), который соответствует времени `slantRangeTime`. Вы можете просто использовать его, выполнив интерполяцию на сетку ваших пикселей.
*   Если вы реализуете процессор с нуля: Вам нужно вычислить $\theta_{GV}$ из времени запаздывания $\tau$, высоты орбиты $H$ и радиуса Земли $R_e$.

#### Этап 4: Преобразование координат (Самый важный момент!)

У нас есть ДНА в координатах антенны ($\theta_{AM}$) и пиксели в координатах Земли ($\theta_{GV}$). Чтобы их сопоставить, нужно учесть крен.

Связь между углами описывается формулой (из PDF, раздел 5.4):
$$\theta_{GV} = \theta_{AM} + \theta_{roll}$$

Следовательно, чтобы найти значение ДНА для пикселя с углом $\theta_{GV}$, нам нужно найти соответствующий угол в антенне:
$$\theta_{target\_ant} = \theta_{GV} - \theta_{roll}$$

**Внимание**: $\theta_{roll}$ может быть разным для разных азимутальных строк (время $\eta$), поэтому это вектор или скаляр, зависящий от текущей строки обработки.

#### Этап 5: Интерполяция и применение коррекции

1.  **Интерполяция**:
    У вас есть вектор ДНА на сетке $\theta_{AM}$ (шаг 0.05 град). Вам нужно получить значение ДНА для каждого пикселя, у которого угол $\theta_{target\_ant}$ (обычно эти углы не совпадают точно с сеткой ДНА).
    
    *   **Критично**: Интерполяцию нужно проводить в **комплексной плоскости**. Не интерполируйте амплитуду и фазу по отдельности, чтобы избежать разрывов фазы.
    *   Используйте линейную интерполяцию действительной и мнимой части:
        $$G_{interp} = \text{InterpolateComplex}(G_{eap\_vector}, \theta_{AM\_grid}, \theta_{target\_ant})$$
    
2.  **Коррекция**:
    Примените деление к каждому пикселю изображения:
    $$s_{corr} = \frac{s}{G_{interp}}$$

    Если разложить на составляющие:
    *   Амплитуда пикселя делится на $|G_{interp}|$ (радиометрическая коррекция).
    *   Фаза пикселя уменьшается на $\arg(G_{interp})$ (фазовая коррекция).

---

### 4. Полезные пояснения для понимания

*   **Почему Phase Correction так важна?**
    В документе `Sentinel-1-IPF_EAP_Phase_correction` подчеркивается, что игнорирование фазовой составляющей ДНА (как было в старых версиях ПО IPF) приводит к "плавающему" фазовому сдвигу между под-swath'ами (например, IW1 и IW2). Это делает невозможной корректную интерферометрию. Используя комплексную ДНА ($I, Q$), вы устраняете этот сдвиг.

*   **Особенности Roll Steering Law**:
    Угол крена не постоянен. Он зависит от высоты спутника, которая меняется по орбите (эллиптичность) и со временем (снижение орбиты). Использование точного закона крена (функции от времени ANX) обязательно для точного совпадения углов.

*   **Центр вектора ДНА**:
    При построении вектора углов $\theta_{AM}$ убедитесь, что вы правильно находите "ноль". Обычно массив ДНА нечетной длины, и центральный элемент соответствует 0 (boresight), либо задается смещение.

*   **Структура данных**:
    Поскольку ДНА зависит от поляризации и полосы (swath), убедитесь, что вы выбираете правильный массив `values` из `calibrationParamsList`. Для IW режима нужно будет обрабатывать каждый под-swath (IW1, IW2, IW3) со своей ДНА.

### Итоговое резюме логики (Workflow)

1.  **Инициализация**: Загрузите XML `aux_cal`. Для текущего swath/polar постройте массив `angles_AM` и массив `complex_EAP`.
2.  **Цикл по Азимуту (строкам)**:
    a. Рассчитайте `theta_roll` для текущего времени строки.
    b. Рассчитайте или получите из метаданных массив `theta_GV` для всех диапазонов этой строки.
    c. Вычислите `theta_ant_array = theta_GV - theta_roll`.
3.  **Цикл по Дальности (пикселям в строке)**:
    a. Для текущего `theta_ant` найдите значение в массиве `complex_EAP` с помощью **комплексной интерполяции**.
    b. Разделите комплексное значение пикселя изображения на полученное значение ДНА.
4.  **Результат**: Скорректированное изображение, свободное от искажений антенны.

Следуя этой структуре и внимательно относясь к интерполяции комплексных чисел и преобразованию систем координат (вычитание roll), вы сможете точно реализовать алгоритм.



Давайте разберем Этап 2 (Roll Steering) максимально подробно. Это действительно самая математически насыщенная часть, так как она связывает *время съемки* с *пространственным положением спутника*, чтобы определить, как именно была повернута антенна.

Логика построена как цепочка:
**Время (t) $\rightarrow$ Высота орбиты (H) $\rightarrow$ Угол крена ($\theta_{roll}$)**

Нам нужно объединить **Раздел 7.2** (время $\rightarrow$ высота) и **Раздел 7.1** (высота $\rightarrow$ крен).

---

### Шаг 1: Определение времени относительно орбиты (Входные данные)

Формулы в секции 7.2 используют время $t$, прошедшее с момента пересечения спутником восходящего узла орбиты (Ascending Node Crossing, ANX).

Вам понадобятся две метки времени из аннотаций продукта (manifest.xml или .SAFE):
1.  `ascendingNodeTime` ($t_{ANX}$): Время, когда спутник пересек экватор, двигаясь с юга на север.
2.  `azimuthTime` ($t_{burst}$): Время начала обрабатываемого азимутального блока (бурста) или строки.

**Формула времени:**
$$t = t_{burst} - t_{ANX}$$

*Результат $t$ будет в секундах. Это то самое "elapsed seconds from the ANX", которое используется в коде.*

---

### Шаг 2: Расчет высоты (H) по времени (Секция 7.2)

Орбита Sentinel-1 не идеальная окружность, она немного эллиптическая. Чтобы узнать точную высоту в конкретный момент времени, используется сумма нескольких синусоид (гармоник).

**Уравнение из документа:**
$$H(t) = h_0 + \sum_{i=0}^{3} h_i \cdot \sin((i+1)\cdot \omega_{orb} \cdot t + \phi_i)$$

Разберем переменные (все значения взяты из PDF, секция 7.2):

**Константы орбиты:**
*   $T_{orb}$ (Период обращения) = $\frac{12 \text{ дней} \times 24 \text{ ч} \times 3600 \text{ с}}{175 \text{ оборотов}} \approx 5924.57 \text{ сек}$.
*   $\omega_{orb}$ (Угловая скорость орбиты) = $\frac{2\pi}{T_{orb}} \approx 0.00106 \text{ рад/с}$.

**Параметры высоты:**
*   $h_0 = 707714.8$ метров (средняя высота).
*   Вектор амплитуд гармоник $h = [8351.5, 8947.0, 23.32, 11.74]$ метров.
*   Вектор фаз гармоник $\phi = [3.1495, -1.5655, -3.1297, 4.7222]$ радиан.

**Алгоритм расчета высоты:**
Для каждого $t$ (из Шага 1) выполняем цикл сложения 4-х синусоид:

1.  Вычисляем аргумент первой гармоники: $\theta_1 = 1 \cdot \omega_{orb} \cdot t + \phi_0$
2.  Вычисляем значение: $val_1 = h_0 \cdot \sin(\theta_1)$  *(Внимание: здесь используется $h[0]$ из массива)*
3.  Вычисляем аргумент второй гармоники: $\theta_2 = 2 \cdot \omega_{orb} \cdot t + \phi_1$
4.  Вычисляем значение: $val_2 = h_1 \cdot \sin(\theta_2)$  *(Используем $h[1]$)*
5.  ... и так далее для $i=2$ и $i=3$.
6.  Складываем всё вместе и прибавляем базовую высоту $h_{base}$ ($707714.8$ м).

В итоге вы получите высоту $H$ в **метрах**.

---

### Шаг 3: Расчет угла крена (Секция 7.1)

Зная точную высоту $H$, мы можем вычислить номинальный угол крена. Суть закона крена: чем выше спутник, тем сильнее нужно "наклониться" вниз (увеличить крен), чтобы луч оставался перпендикулярным поверхности.

**Формула из документа:**
$$\theta_{roll} = \theta_{ref} - \alpha_{roll} \cdot (H - H_{ref})$$

Разберем переменные (все значения взяты из PDF, секция 7.1):

*   $\theta_{ref} = 29.450$ градусов (Угол крена на референсной высоте).
*   $H_{ref} = 711.700$ **км** (Референсная высота).
*   $\alpha_{roll} = 0.05660$ град/км (Чувствительность крена к высоте).

**Алгоритм расчета угла крена:**
1.  Переведите высоту $H$ из метров в **километры** (т.к. коэффициенты даны в км): $H_{km} = H / 1000$.
2.  Вычислите разницу высот: $\Delta H = H_{km} - H_{ref}$.
3.  Умножьте разницу на чувствительность: $correction = \alpha_{roll} \cdot \Delta H$.
4.  Вычтите поправку из референсного угла: $\theta_{roll} = \theta_{ref} - correction$.

Результат $\theta_{roll}$ будет в **градусах**.

---

### Практический пример (Псевдокод)

Вот как это выглядит в виде кода, чтобы вы могли проверить свою реализацию.

```python
import math

# --- КОНСТАНТЫ (Из PDF секции 7.1 и 7.2) ---
T_orb = (12 * 24 * 3600) / 175.0  # Период орбиты, сек ~5924.57
w_orb = 2 * math.pi / T_orb        # Угловая скорость

# Параметры высоты (Гармоники)
h_base = 707714.8                  # Метры
h_amps = [8351.5, 8947.0, 23.32, 11.74]     # Метры
phases = [3.1495, -1.5655, -3.1297, 4.7222] # Радианы

# Параметры закона крена
roll_ref = 29.450                  # Градусы
height_ref_km = 711.700            # Км
alpha_roll = 0.05660               # Град/км

def calculate_roll_angle(azimuth_time, anx_time):
    """
    azimuth_time: время начала строки/бурста (в секундах UTC)
    anx_time: время восходящего узла (в секундах UTC)
    Возвращает: Угол крена в градусах
    """
    
    # ШАГ 1: Время относительно ANX
    t = azimuth_time - anx_time
    
    # ШАГ 2: Расчет высоты H(t)
    sum_harmonics = 0
    for i in range(4):
        # Обратите внимание: (i+1) для частоты, индексы массивов 0..3
        freq = (i + 1) * w_orb
        phase = phases[i]
        amp = h_amps[i]
        
        # Синусоида
        sum_harmonics += amp * math.sin(freq * t + phase)
        
    current_height_m = h_base + sum_harmonics
    current_height_km = current_height_m / 1000.0
    
    # ШАГ 3: Расчет угла крена
    # Разница высот относительно референса
    delta_h = current_height_km - height_ref_km
    
    # Линейная зависимость
    roll_angle = roll_ref - alpha_roll * delta_h
    
    return roll_angle

# ПРИМЕР ИСПОЛЬЗОВАНИЯ:
# Допустим, anx_time = 10000.0 сек
# azimuth_time = 11500.0 сек (спустя 1500 сек после ANX)
# roll = calculate_roll_angle(11500.0, 10000.0)
# print(roll) 
# Должно получиться значение порядка 29.xx градусов
```

### На что обратить внимание (Common Pitfalls)

1.  **Единицы измерения**: Самая частая ошибка — перепутать метры и километры. В секции 7.1 формула ожидает разницу высот в **километрах**, так как коэффициент $\alpha_{roll}$ мал (0.05). Если вы подадите метры, получите нереальный крен в тысячи градусов.
2.  **Индексация в цикле**: В уравнении используется $(i+1)\omega$. Не забудьте множитель $(i+1)$. Для первого элемента массива это $1 \cdot \omega$, для второго $2 \cdot \omega$ и т.д.
3.  **Область определения**: Угол крена $\theta_{roll}$ — это угол наклона самого спутника.
    *   В формулах Этапа 3 (коррекции EAP) мы использовали:
        $$\theta_{target\_ant} = \theta_{GV} - \theta_{roll}$$
    *   Убедитесь, что вы используете полученный здесь $\theta_{roll}$ именно в этом уравнении. Знак минус здесь критичен: это переход из системы координат "Земля/Вид" ($\theta_{GV}$) в систему координат "Антенна" ($\theta_{ant}$).

Теперь, имея эту функцию `calculate_roll_angle`, вы можете вычислять угол крена для каждой азимутальной строки (блока) вашего изображения и использовать его для поиска правильного значения в таблице EAP.



Это отличный вопрос, потому что здесь легко запутаться в иерархии файлов и типах времени. Давайте расставим всё по местам.

Вам нужно два значения времени, которые вычтите друг из друга:
$$t = \text{azimuthTime} - \text{ascendingNodeTime}$$

Вот откуда брать каждое из них и как их интерпретировать:

---

### 1. `ascendingNodeTime` (Время восходящего узла)

Это глобальная точка отсчета для всей орбиты. Для всего продукта (IW1, IW2, IW3) это значение будет **одинаковым**.

*   **Откуда брать:** Из файла `manifest.safe`.
*   **Путь (XPATH):** `/safe:manifest/safe:extension/s1:orbitProperties/s1:ascendingNodeTime`
*   **Значение:** `2025-10-25T17:11:18.931874`

Это строка в формате ISO 8601. Для вычислений вам нужно превратить её в **секунды** (например, в Unix Timestamp или в секунды с начала эпохи).

---

### 2. `azimuthTime` (Азимутальное время)

Здесь есть нюанс. Весь продукт состоит из нескольких "бурстов" (bursts). Угол крена (Roll) изменяется со временем. Поэтому брать одно время на весь файл неверно. Вам нужно брать `azimuthTime` **для каждого обрабатываемого бурста**.

*   **Откуда брать:** Из файла аннотации (XML) конкретного под-swath'a, который вы сейчас обрабатываете (например, `s1a-iw1-slc-vv-...xml`).
*   **Путь (XPATH):** `/product/swathTiming/burstList/burst/azimuthTime`
*   **Значение:** Это время **начала** конкретного бурста.

Пример из вашей структры файла:
У вас в файле `s1a-iw1-slc-vv-20251025t172519-...` (имя файла подсказывает время начала 17:25:19) внутри XML будет тег:
```xml
<swathTiming>
    <burstList>
        <burst>
            <azimuthTime>2025-10-25T17:25:19.987654</azimuthTime>
            ...
```

Если у вас в продукте несколько бурстов (например, 9), то для:
*   Бурста 1: $t_{burst1}$ = `2025-10-25T17:25:19.xxx`
*   Бурста 2: $t_{burst2}$ = `2025-10-25T17:25:46.xxx`
*   ... и так далее.

**Важно:** Для алгоритма EAP Correction вы рассчитываете угол Roll именно для времени начала этого конкретного бурста. Поскольку бурст длится всего около 2.5 секунд (в режиме IW), можно считать, что угол крена неизменен в течение всего этого бурста.

---

### 3. Как превратить строки в секунды (Пример)

Оба времени даны в строковом формате с точностью до долей секунды. Вам нужно привести их к одной системе отсчета (например, `double` секунды с начала эпохи Unix или просто разница между двумя датами).

Алгоритм действий:

1.  Взять строку `2025-10-25T17:11:18.931874` ($t_{ANX}$).
2.  Взять строку `2025-10-25T17:25:19.123456` ($t_{burst}$).
3.  Преобразовать обе в **UTC time**.
4.  Вычислить разницу: $\Delta t = t_{burst} - t_{ANX}$.

**Пример вычислений (вручную):**
*   $t_{ANX}$: 17:11:18.93...
*   $t_{burst}$: 17:25:19.12...
*   Разница $\approx$ 14 минут 1 секунда $\approx$ 841 секунда.

Именно это значение (около 841 сек) вы подставите в формулу из раздела 7.2 в переменную `t`.

---

### Итоговая таблица для вашего кода

| Переменная | Источник файла | XML-путь | Примечание |
| :--- | :--- | :--- | :--- |
| **`t_ANX`** | `manifest.safe` | `/s1:orbitProperties/s1:ascendingNodeTime` | Одно для всего файла (для IW1, IW2, IW3) |
| **`t_burst`** | `measurement/...-iw1-...xml` | `/product/swathTiming/burstList/burst/azimuthTime` | Уникальное для каждого бурста внутри под-swath'а |

**Резюме:**
1.  Парсите `ascendingNodeTime` из `manifest.safe` **один раз**.
2.  В цикле по бурстам (или при обработке каждого бурста) берите `azimuthTime` из соответствующего XML аннотации.
3.  Вычитайте, получайте секунды и скармливайте функции `calculate_roll_angle`.



Это очень важный момент, который часто вызывает путаницу. Ответ на ваш вопрос может вас удивить, но он строго следует из документа **DI-MPC-IPFDPM**.

**Краткий ответ: Вы НЕ должны склеивать матрицы для IW1 (Swath 10), IW2 (Swath 11) и IW3 (Swath 12) в одну большую матрицу.**

### Почему их нельзя просто взять и склеить?

Согласно документу (раздел **7.3.2 Burst Merging (TOPSAR GRD Only)**):
> "Burst merging... is performed to combine data from different sub-swaths... This step is **only applicable to GRD** products."
> *(Слияние бурстов... выполняется для объединения данных из разных под-swath'ов... Этот этап применим **только к GRD** продуктам).*

Для SLC продуктов (которые вы, похоже, обрабатываете, судя по "свертке" и "комплексному сигналу") **под-swath'ы (IW1, IW2, IW3) хранятся и обрабатываются как независимые компоненты**.

Почему нельзя просто склеить ваши матрицы (1409, 1548, 1410):
1.  **Разная частота дискретизации по дальности:** У каждого под-swath'а своя `rangeSamplingRate`.
2.  **Разная геометрия:** IW1 смотрит ближе, IW3 дальше. Пространственное разрешение в метрах на пиксель разное, хотя в SLC (Slant Range) шаг по дальности (time) фиксирован для каждого свата, но разный между ними.
3.  **Разные размерности:** Именно поэтому у вас матрицы разной ширины.

### Что же тогда нужно делать?

Вам нужно не "склеивать сваты друг с другом", а **собирать (сшивать) БУРСТЫ внутри каждого свата**.

**Размерность (1409, 23808)** для Swath 10 — это, скорее всего, результат того, что вы обработали 9 бурстов, но не "слепили" их правильно или взяли не все строки. Один бурст в IW режиме обычно содержит около 2500 строк. 1409 — это либо половина бурста, либо специфический выбор.

Вам нужно выполнить "Mosaicking" (Мозаичное объединение) **бурстов одного свата**.

---

### Какой параметр времени искать и использовать (согласно документу)

Чтобы отсортировать ваши "разрозненные" файлы (.pkl) и собрать их в правильном порядке, а также отнести нужную строку к нужному бурсту, вы должны использовать параметр **`azimuthTime`** (Азимутальное время).

Согласно **Section 6.4.1 Azimuth Post-Processing (TOPSAR only)** и **Section 8.2.2 Internal Signal Data Slice Definition**:
> The internal signal data slices are generated by mosaicking range compressed data blocks into bursts.

**Алгоритм сборки:**

1.  **Из XML (metadata):** Вы берете список бурстов. Для каждого бурста есть точное время начала:
    *   Burst 1: `azimuthTime = T1`
    *   Burst 2: `azimuthTime = T2`
    *   Burst 3: `azimuthTime = T3`
2.  **Из ваших .pkl файлов:** Вы должны брать время строки (или время захвата, которое вы там сохранили). Назовем его `line_time`.
3.  **Правило отбора:** Строка принадлежит Бурсту 1, если:
    *   `T_start_Burst1 <= line_time < T_end_Burst1`
    *   Обычно Burst'ы следуют друг за другом без перекрытия или с небольшим перекрытием. Сортировка по `azimuthTime` автоматически решает задачу.

### Как правильно сшить матрицы внутри одного Swath (например, Swath 10)

Допустим, вы собрали данные для Swath 10. У вас должно получиться несколько "пачек" данных (бурстов).

Синтаксически, чтобы получить финальную матрицу Swath 10 (например, размер 25000 x 23808), вы делаете так:

1.  **Сортировка:** Убедитесь, что ваши бурсты (матрицы) отсортированы по возрастанию `azimuthTime`.
2.  **Объединение (Stacking):** Вы "накладываете" их друг на друга по вертикали (по оси 0 — оси Азимута).

Пример на Python:

```python
# Допустим, мы собрали список матриц бурстов для Swath 10 (IW1)
# Список: [burst_matrix_1, burst_matrix_2, ..., burst_matrix_9]
# Размер каждого: например, (2500, 23808)

# 1. Проверяем сортировку по времени (burst_meta из XML)
# ...
# 2. Объединяем (конкатенация по оси 0)
final_swath_matrix = np.concatenate([burst_matrix_1, burst_matrix_2, ...], axis=0)

# Результат будет иметь размер: (2500*9, 23808) = (22500, 23808)
```

### Резюме

1.  **Не склеивайте Swath 10, 11, 12 между собой.** Они разные продукты. Обрабатывайте их независимо друг от друга.
2.  **Внутри Swath 10 (или 11, 12):** Склеивайте (конкатенируйте) только Бурсты (Bursts) друг с другом по времени.
3.  **Ключевой параметр времени:** Используйте **`azimuthTime`** из тега `<burst>` в XML-аннотации. Это время начала бурста.
    *   Сопоставляйте строки из pkl с этим временем.
    *   Если `время_строки` находится между `azimuthTime` текущего бурста и следующим бурстом — значит строка относится к текущему.
    *   Сортируйте строки по этому времени, прежде чем формировать матрицу.

Если вы попробуете склеить Swath 10 и Swath 11 напрямую (`np.hstack` или `np.vstack`), вы получите артефакты на стыке, так как координатные сетки по дальности (Range) у них не совпадают.



Да, вы абсолютно правы, и очень хорошо, что вы задали этот вопрос, чтобы структурировать данные правильно.

### Ответ на ваш вопрос:
1.  **Да, служебная информация (metadata) критически важна и используется.** Без нее данные бесполезны для точной обработки.
2.  **Да, структура именно такая:** У вас есть **Матрица данных** и параллельно ей **Список словарей**, где индекс списка `i` соответствует строке `i` матрицы.

Давайте разберем, зачем она нужна и как устроена связь.

---

### 1. Зачем нужна эта служебная информация (Metadata)?

Матрица `slc_matrix` содержит только **Амплитуду и Фазу** (сигнал). Но чтобы сделать из этого **Изображение** или применить калибровку (EAP), процессору нужно знать: **"Где и Когда эта строка была записана?"**.

Вам понадобятся следующие поля из `meta_list[i]`:

#### А. `datationService` (Самое важное сейчас)
*   **Зачем нужно:** Чтобы сгруппировать строки в Бурсты (Bursts).
*   **Как используется:** Вы декодируете байты в `datetime`. Потом смотрите в XML: "Ого, эта строка записана в 17:25:20, значит, она относится к Бурсту №2".

#### Б. `SWST` (Sampling Window Start Time)
*   **Зачем нужно:** Это время начала "окна" считывания по дальности.
*   **Как используется:** Это нужно для точного расчета геометрии (Range Time). Если в пакете есть сдвиг по дальности (например, из-за изменения высоты), без `SWST` вы не сможете точно привязать пиксель к географической координате.

#### В. `fs` (Sampling Frequency)
*   **Зачем нужно:** Частота дискретизации.
*   **Как используется:** Чтобы рассчитать вектор времени по дальности: $t[i] = t[0] + i/fs$.

---

### 2. Как выглядит эта структура (Визуализация)

Представьте это как две полосы, которые лежат друг над другом. Нумерация строк в них полностью совпадает.

```python
# Индекс (строка):
# 0      ->  [Комплексные данные строки 0]     <->  { 'time': ..., 'swst': ... }
# 1      ->  [Комплексные данные строки 1]     <->  { 'time': ..., 'swst': ... }
# 2      ->  [Комплексные данные строки 2]     <->  { 'time': ..., 'swst': ... }
# ...
# 1547   ->  [Комплексные данные строки 1547]  <->  { 'time': ..., 'swst': ... }
```

Если вы возьмете строку матрицы `slc_matrix[:, 5]`, то metadata для этой строки — это элемент `meta_list[5]`.

### Почему нельзя хранить всё в одной матрице?

NumPy-массивы работают быстро только тогда, когда все элементы одного типа (например, `complex128` или `float32`).
Словари (Dict) — это сложные объекты. Если вы попытаетесь создать матрицу объектов словарей (`dtype=object`), NumPy потеряет всю свою производительность и скорость операций (умножение, сложение массивов).

Поэтому подход "Матрица + Список" — это стандартный профессиональный способ.
*   **Матрица** — для быстрых вычислений (свертка, фильтрация, деление на корректирующую матрицу).
*   **Список** — для логики (сортировка, группировка, поиск).

### Что будет дальше?

Когда вы перейдете к следующему этапу — **Группировке в Бурсты** — вы будете использовать эту структуру вот так:

```python
# slc_matrix: (Range_Samples, Azimuth_Lines)
# meta_list:   [Dict_0, Dict_1, ..., Dict_N]

# Пример: фильтруем строки, принадлежащие конкретному бурсту
burst_lines_indices = []

for i, meta in enumerate(meta_list):
    line_time = decode_time(meta['datation_service'])
    
    # Проверяем, попадает ли line_time в интервал Бурста (из XML)
    if is_time_in_burst(line_time, burst_start_time):
        burst_lines_indices.append(i)

# Теперь у нас есть индексы строк, которые принадлежат этому бурсту.
# Вырезаем их из матрицы:
burst_matrix = slc_matrix[:, burst_lines_indices]

# И вырезаем метаданные (хотя для самого калькулятора EAP они уже не нужны, 
# так как мы используем глобальное время бурста, но они были нужны, чтобы найти burst_matrix)
```

**Резюме:** Вы всё поняли правильно. Матрица — это "Мясо" (сигнал), а список `meta_list` — это "Опознавательные знаки" (привязка ко времени и пространству), которые позволяют правильно собрать это мясо в конечное блюдо (Бурст).